# Imports

In [1]:
import numpy as np
import pandas
import pandas as pd
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, normalize
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
import json
import time
import seaborn as sns
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from scipy import stats
simplefilter("ignore", category=ConvergenceWarning)

## Import Data and begin cleaning

In [2]:
path1 = 'dirty_data_large.csv' #name filepath
json_cols = ['metadata'] #metadata is hidden in json structure within a column of the data
df = pd.read_csv(path1) #read dirty data into a pandas dataframe
df['createdAt'] = pd.to_datetime(df['createdAt']) #enfore date-time data type
df['last_sale_price'] = df.sort_values(by='createdAt', ascending=False).groupby(['project','tokenId'])['listing_usd_price'].shift(-1) #find the last sale price of every NFT that has more than one transaction
df = pd.get_dummies(df, columns=['project']) #Binarize the categorical project variable using one-hot encoding

In [3]:
df['price_of_eth'] = df['listing_usd_price'] / df['listing_eth_price'] #get current price of eth
df['date_delta'] = (df['createdAt'] - df['createdAt'].min())  / np.timedelta64(1,'D') #find the time since the earliest transaction to pick up trends

Look at data to see if it matches our expectations

In [5]:
df.head()

,_id,image_url_BIG,metadata,rarity,rarityPercentile,rarityRank,tokenId,createdAt,duration,endingPrice,...,project_boredapeyachtclub,project_cool-cats-nft,project_creatureworld,project_deadfellaz,project_lazy-lions,project_pudgypenguins,project_robotos-official,project_world-of-women-nft,price_of_eth,date_delta
0,615710a4b2fe4c479134364e,https://lh3.googleusercontent.com/jw_je5EhXyyS...,"[{""key"":""Fur"",""value"":""Black""},{""key"":""Backgro...",31.036050,97.12,289.0,246,2021-09-24 04:43:44+00:00,NaN,NaN,...,1,0,0,0,0,0,0,0,4240.92,8.950299
1,615710a6b2fe4c4791354a8f,https://lh3.googleusercontent.com/N8nIM0rKHw3R...,"[{""key"":""Background"",""value"":""New Punk Blue""},...",2.159267,26.46,7355.0,261,2021-09-23 02:03:33.020000+00:00,NaN,NaN,...,1,0,0,0,0,0,0,0,4255.78,7.839061
2,615710a7b2fe4c479135b467,https://lh3.googleusercontent.com/ZPSrGy7OeeFs...,"[{""key"":""Fur"",""value"":""Black""},{""key"":""Earring...",6.840881,77.47,2254.0,285,2021-09-24 04:41:40.101000+00:00,NaN,NaN,...,1,0,0,0,0,0,0,0,4238.79,8.948865
3,615710a7b2fe4c479135b467,https://lh3.googleusercontent.com/ZPSrGy7OeeFs...,"[{""key"":""Fur"",""value"":""Black""},{""key"":""Earring...",6.840881,77.47,2254.0,285,2021-09-24 04:35:53.545000+00:00,NaN,NaN,...,1,0,0,0,0,0,0,0,4238.79,8.944854
4,615710a7b2fe4c479135b467,https://lh3.googleusercontent.com/ZPSrGy7OeeFs...,"[{""key"":""Fur"",""value"":""Black""},{""key"":""Earring...",6.840881,77.47,2254.0,285,2021-09-24 04:41:18.144000+00:00,NaN,NaN,...,1,0,0,0,0,0,0,0,4240.92,8.948611


Check that the last sale price looks accurate

In [11]:
df[(df['project_lazy-lions']==1) & (df['tokenId']==200)][['project_lazy-lions','tokenId','createdAt','last_sale_price','listing_usd_price']]

,project_lazy-lions,tokenId,createdAt,last_sale_price,listing_usd_price
115064,1,200,2021-09-22 03:43:32.226000+00:00,NaN,631.2480
115065,1,200,2021-09-22 10:47:45.368000+00:00,631.2480,596.0388
115066,1,200,2021-09-22 12:57:02.205000+00:00,596.0388,341.5408


Great. Everything looks in perfect order. Now let's unpack the metadata.

This will take a while.

In [4]:
#Function to unwrap meta data hidden in json
def clean_json2(x):

    # store values
    ls = []
    datahouse = json.loads(x[0])
    # loop through the list f dictionaries
    for y in range(len(datahouse)):

        # Access each key and value in each dictionary
        for k, v in datahouse[y].items():
            # append column names to ls
            ls.append(str(k)+ "_" +str(v))

    # create a new column or change 0 to 1 if keyword exists
    for z in range(len(ls)):

        # If column not in the df columns then make a new column and assign zero values while changing the current row to 1
        if ls[z] not in df.columns:
            df[ls[z]] = 0
            df[ls[z]].iloc[x.name] = 1
        else:
            df[ls[z]].iloc[x.name] = 1
    return

In [12]:
for x in json_cols:
    df[[x]].apply(clean_json2, axis=1)

## Export Data so that it can be used for modeling / visualization

In [13]:
#Data seems clean enough
path2 = 'clean_data_large.csv'
df.to_csv(path2)